In [0]:
# https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

In [0]:
word_to_idx = {'hello': 0, 'world': 1}
embeds = nn.Embedding(2,5)
lookup_tensor = torch.tensor([word_to_idx['hello']], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229]],
       grad_fn=<EmbeddingBackward>)


In [0]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocab)}


[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [0]:
class NGramLanguageModeler(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super(NGramLanguageModeler, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(context_size*embedding_dim, 128)
    self.linear2 = nn.Linear(128, vocab_size)

  
  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1, -1))
    out = F.relu(self.linear1(embeds))
    out = self.linear2(out)
    log_probs = F.log_softmax(out, dim=1)
    return log_probs

In [0]:
losses = []
loss_fn = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [0]:
for epoch in range(10):
  total_loss = 0
  for context, target in trigrams:
    context_ids = torch.tensor([word_to_idx[w] for w in context], dtype=torch.long)
    model.zero_grad()
    log_probs = model(context_ids)
    loss = loss_fn(log_probs, torch.tensor([word_to_idx[target]], dtype=torch.long))
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  losses.append(total_loss)


In [0]:
print(losses)

[519.8471801280975, 517.4835438728333, 515.1353075504303, 512.8006122112274, 510.47846722602844, 508.16847920417786, 505.87068223953247, 503.58320116996765, 501.3050982952118, 499.0369007587433]


In [0]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()
vocab = set(raw_text)
vocab_size = len(vocab)

In [0]:
word_to_idx = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [0]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, embedding_size, context_size):
    super(CBOW, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(context_size*context_size*embedding_dim, 128)
    self.linear2 = nn.Linear(128, vocab_size)

  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((1, -1))
    out = F.relu(self.linear1(embeds))
    out = self.linear2(out)
    log_probs = F.log_softmax(out, dim=1)
    return log_probs

In [0]:
losses = []
loss_fn = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE*CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [0]:
for epoch in range(10):
  total_loss = 0
  for context, target in data:
    context_ids = torch.tensor([word_to_idx[w] for w in context], dtype=torch.long)
    model.zero_grad()
    log_probs = model(context_ids)
    loss = loss_fn(log_probs, torch.tensor([word_to_idx[target]], dtype=torch.long))
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  losses.append(total_loss)

print(losses)

[224.68842387199402, 223.25896453857422, 221.83899545669556, 220.4266812801361, 219.02206325531006, 217.62479996681213, 216.23384761810303, 214.8500201702118, 213.47162985801697, 212.09855222702026, 210.730406999588, 209.36647582054138, 208.0065472126007, 206.64905881881714, 205.29560208320618, 203.94508004188538, 202.59707641601562, 201.25115036964417, 199.90720391273499, 198.56374835968018]
